Analyse Candlestick of *AAPL*

STEP 0. Import candlestick data from finnhub

In [ ]:
pip install finnhub-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import finnhub
finnhub_client = finnhub.Client(api_key="cfgveo1r01qlga2um3tgcfgveo1r01qlga2um3u0")

#candle ticker,time period,unix time stamp from, to
#Result c:close price, h:high price,L: low price, o: open price
print(finnhub_client.stock_candles('AAPL', 'D', 1325307600, 1672462800))

{'c': [14.68677142857143, 14.765699999999999, 14.929628571428571, 15.085700000000001, 15.06177142857143, 15.1157, 15.091060714285714, 15.04962857142857, 14.9932, 15.167839285714285, 15.325339285714287, 15.276771428571427, 15.0107, 15.264589285714285, 15.014628571428572, 15.952128571428572, 15.87962857142857, 15.974271428571429, 16.178910714285713, 16.302839285714285, 16.29247857142857, 16.25427142857143, 16.41712857142857, 16.570339285714287, 16.743910714285715, 17.024271428571428, 17.6132, 17.622128571428572, 17.94997857142857, 18.19497857142857, 17.773910714285716, 17.93605, 17.932839285714287, 18.38747857142857, 18.322839285714284, 18.442478571428573, 18.657478571428573, 18.77712142857143, 19.121771428571428, 19.372839285714285, 19.445339285714287, 19.470689285714286, 19.041410714285714, 18.937839285714286, 18.953200000000002, 19.356771428571427, 19.470339285714285, 19.71427142857143, 20.28927142857143, 21.056410714285715, 20.912839285714284, 20.913189285714285, 21.467839285714287, 

In [ ]:
import pandas as pd
import finnhub
import datetime
finnhub_client = finnhub.Client(api_key="cfgveo1r01qlga2um3tgcfgveo1r01qlga2um3u0")
data = finnhub_client.stock_candles('AAPL', 'D', 1325307600, 1672462800)
df = pd.DataFrame(data)
df = df.drop('s', axis=1)
df = df.rename(columns={'c': 'Close','h':'High','l':'Low','o':"Open",'t':"TimeStamp",'v':'Volume'})
df['Date'] = pd.to_datetime(df['TimeStamp'], unit='s')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df

,Close,High,Low,Open,TimeStamp,Volume
Date,,,,,,
2012-01-03,14.686771,14.732129,14.607129,14.621411,1325548800,302259076
2012-01-04,14.765700,14.809989,14.617129,14.642839,1325635200,260244684
2012-01-05,14.929629,14.948200,14.738200,14.819629,1325721600,271267472
2012-01-06,15.085700,15.098200,14.972129,14.991771,1325808000,318385956
2012-01-09,15.061771,15.276771,15.048200,15.196421,1326067200,394023560
...,...,...,...,...,...,...
2022-12-23,131.860000,132.415000,129.640000,130.920000,1671753600,63814893
2022-12-27,130.030000,131.410000,128.720000,131.380000,1672099200,69007830
2022-12-28,126.040000,131.027500,125.870000,129.670000,1672185600,85438391


### STEP 1.1 (Basic Version) Identify the bullish_engulfing patterns & Calculate its returns in 10-D 

In [ ]:
def get_pattern(row, prev_row):
    if row['Open'] < row['Close']:
        if row['Open'] <= prev_row['Close'] and row['Close'] > prev_row['Open'] and (row['Open']-row['Low'])>(row['High']-row['Close']) and (row['Close'] - row['Open']) / (row['High'] - row['Low']) >= 0.7:
            return "Bullish Engulfing"   #a sign that the price would go up
        elif (row['Close'] - row['Open']) / (row['High'] - row['Low']) >= 0.6:
            return "Bullish"
        else:
            return "Neutral"
    elif row['Open'] > row['Close']:
        if row['Close'] < prev_row['Open'] and row['Open'] >= prev_row['Close']and (row['Close']-row['Low'])<(row['High']-row['Open']) and (row['Open'] - row['Close']) / (row['High'] - row['Low']) >= 0.7:
            return "Bearish Engulfing"   #a sign that the price would go down
        elif (row['Open'] - row['Close']) / (row['High'] - row['Low']) >= 0.6:
            return "Bearish"
        else:
            return "Neutral"
    else:
        return "Doji"

# Apply the function to each row in the DataFrame to create a new "Pattern" column
prev_row = None
patterns = []
for index, row in df.iterrows():
    if prev_row is None:
        prev_row = row
        continue
    patterns.append(get_pattern(row, prev_row))
    prev_row = row
df['Pattern'] = ["Neutral"] + patterns

# Print the first 5 rows of the updated DataFrame
print(df.head())

                Close       High        Low       Open   TimeStamp     Volume  \
Date                                                                            
2012-01-03  14.686771  14.732129  14.607129  14.621411  1325548800  302259076   
2012-01-04  14.765700  14.809989  14.617129  14.642839  1325635200  260244684   
2012-01-05  14.929629  14.948200  14.738200  14.819629  1325721600  271267472   
2012-01-06  15.085700  15.098200  14.972129  14.991771  1325808000  318385956   
2012-01-09  15.061771  15.276771  15.048200  15.196421  1326067200  394023560   

            Pattern  
Date                 
2012-01-03  Neutral  
2012-01-04  Bullish  
2012-01-05  Neutral  
2012-01-06  Bullish  
2012-01-09  Neutral  


In [ ]:
# Define a function to calculate the return after 10 trading days
def calculate_return(df, index):
    next_day = index + pd.Timedelta(days=10)
    if not df.index.isin([next_day]).any():
        return None
    return (df.loc[next_day, 'Close'] - df.loc[index, 'Open']) / df.loc[index, 'Open']

# Get the index of the next day of a "Bullish Engulfing" date
be_indices = df[df['Pattern'] == "Bullish Engulfing"].index.tolist()

# Calculate the return after 10 trading days for each "Bullish Engulfing" date
returns = [calculate_return(df, index) for index in be_indices]

# Remove None values from the returns list
returns = [r for r in returns if r is not None]

returns

[0.024807050249860213,
 0.014901548737443211,
 0.07845150293074074,
 0.020840487743473468,
 0.0645314258100906,
 -0.002237098177634199,
 0.03524070999493774,
 0.05668354987888142,
 0.07148114075436979,
 0.014429621562755273,
 0.030593076303783766,
 0.0732890246093048,
 -0.008145615446500425,
 -0.01817690992531266,
 0.038812251578208945,
 0.034338061465721055,
 0.1136730607763353,
 0.03922315308453927,
 -0.0016054309435346741,
 -0.038687856183107175,
 0.035806877659834625,
 0.0170180722891567,
 0.05897351538081923,
 0.0461946511456569,
 0.07036181147173448,
 0.07301133462677419,
 0.019254520695443016,
 0.05712037765538939,
 0.0388289676425269,
 0.05925981987436625,
 0.011518570756934696,
 0.0940128340913339,
 -0.00017424638438753057,
 0.06490738018229943,
 0.09000508093198792,
 -0.04574375460451418]

In [ ]:
# Test: if we have $100 at the beginning of 2012, how much we would have in 11 yr by idnetifying "Bullish Engulfing" as a buying signal.
# Strategy: buy AAPL the day after "Bullish Engulfing", hold it for ten trading days then sell all the position.
original_money = 100

for i in returns:
  original_money = original_money * (1+i)
print("we would have",round(original_money,2),"by the end of 2022.")
print("The yearly return is :",round(((original_money/100)**(1/11)-1)*100,2),"%.")

we would have 362.11 by the end of 2022.
The yearly return is : 12.41 %.


### STEP 1.2 (Advanced) Identify More Single Candlestick Patterns 
i.e 'bullish_hammer', 'bullish_engulfing', 'bullish_kicker', 'bearish_hammer','bearish_engulfing, bearish_kicker

In [ ]:
import finnhub
import pandas as pd

finnhub_client = finnhub.Client(api_key="cfgveo1r01qlga2um3tgcfgveo1r01qlga2um3u0")
data = finnhub_client.stock_candles('AAPL', 'D', 1325307600, 1672462800)
df = pd.DataFrame(data)
df = df.drop('s', axis=1)
df = df.rename(columns={'c': 'close','h':'high','l':'low','o':"open",'t':"TimeStamp",'v':'Volume'})
df['date'] = pd.to_datetime(df['TimeStamp'], unit='s')
df['date'] = pd.to_datetime(df['date'])

#Identify buying signals
# Calculate the bullish hammer candlestick pattern
df['bullish_hammer'] = ((df['open'] - df['high']).abs() <= 0.01 * (df['high'] - df['low'])) & \
               ((df['close'] - df['low']).abs() <= 0.01 * (df['high'] - df['low'])) & \
               ((df['close'] > df['open']) & ((df['close'] - df['open']) > 0.6 * (df['high'] - df['low'])))

# Calculate the bullish engulfing candlestick pattern
df['bullish_engulfing'] = (df['open'].shift(1) > df['close'].shift(1)) & \
                          (df['close'] > df['open']) & \
                          ((df['close'] - df['open']) > (df['open'].shift(1) - df['close'].shift(1)))

# Calculate the bullish kicker candlestick pattern
df['bullish_kicker'] = (df['open'].shift(1) > df['close'].shift(1)) & \
                       (df['open'] < df['close']) & \
                       (df['close'] > df['open'])

#Identify selling signals
# Calculate the bearish hammer candlestick pattern
df['bearish_hammer'] = ((df['high'] - df['open']).abs() <= 0.01 * (df['high'] - df['low'])) & \
                       ((df['close'] - df['low']).abs() <= 0.01 * (df['high'] - df['low'])) & \
                       ((df['open'] > df['close']) & ((df['open'] - df['close']) > 0.6 * (df['high'] - df['low'])))

# Calculate the bearish engulfing candlestick pattern
df['bearish_engulfing'] = (df['open'].shift(1) < df['close'].shift(1)) & \
                          (df['close'] < df['open']) & \
                          ((df['close'] - df['open']) < (df['open'].shift(1) - df['close'].shift(1)))
                
# Calculate the bearish kicker candlestick pattern
df['bearish_kicker'] = (df['open'].shift(1) < df['close'].shift(1)) & \
                       (df['open'] > df['close']) & \
                       (df['close'] < df['open'])

df.head()

,close,high,low,open,TimeStamp,Volume,date,bullish_hammer,bullish_engulfing,bullish_kicker,bearish_hammer,bearish_engulfing,bearish_kicker
0,14.686771,14.732129,14.607129,14.621411,1325548800,302259076,2012-01-03,False,False,False,False,False,False
1,14.765700,14.809989,14.617129,14.642839,1325635200,260244684,2012-01-04,False,False,False,False,False,False
2,14.929629,14.948200,14.738200,14.819629,1325721600,271267472,2012-01-05,False,False,False,False,False,False
3,15.085700,15.098200,14.972129,14.991771,1325808000,318385956,2012-01-06,False,False,False,False,False,False
4,15.061771,15.276771,15.048200,15.196421,1326067200,394023560,2012-01-09,False,False,False,False,True,True


### STEP 2 Test returns using different signal pairs

First, using "bullish_engulfing" close price as the buying point and "bearish_engulfing" close price as the selling point.

In [ ]:
# Filter the dataframe to only include rows with a bullish engulfing pattern
bullish_engulfing_df = df[df['bullish_engulfing'] == True]

# Filter the dataframe to only include rows with a bearish engulfing pattern
bearish_engulfing_df = df[df['bearish_engulfing'] == True]

# Create an empty list to store the returns
returns = []

# Loop through each bullish engulfing pattern
for i in range(len(bullish_engulfing_df)):
    # Find the next bearish engulfing pattern after the bullish engulfing pattern
    next_bearish_engulfing_df = bearish_engulfing_df[bearish_engulfing_df['TimeStamp'] > bullish_engulfing_df.iloc[i]['TimeStamp']].head(1)
    if len(next_bearish_engulfing_df) > 0:
        # Calculate the return from the bullish engulfing closing price to the bearish engulfing opening price
        buy_price = bullish_engulfing_df.iloc[i]['close']
        sell_price = next_bearish_engulfing_df.iloc[0]['close']
        cum_return = (sell_price - buy_price) / buy_price
        returns.append(cum_return)

returns

[0.08264430268386955,
 0.05753158107982658,
 0.05437776220813202,
 0.08038881188864139,
 -0.005489377137561611,
 0.035355065533514626,
 0.015857310393377566,
 -0.008269480076284579,
 -0.08105633694243412,
 -0.007099565769792341,
 -0.019255788376074424,
 0.018600543437108763,
 0.0020330924352486538,
 -0.015766837325589624,
 0.04145535081029196,
 -0.009252643880151786,
 0.003256528346722901,
 -0.016310733471871442,
 -0.04794399743636242,
 0.036998294823598475,
 0.0618218733867692,
 0.018783772658723066,
 -0.018119488083459836,
 -0.024971431235047983,
 -0.07811287279149043,
 -0.06456154836434394,
 -0.05912989852995799,
 -0.06692904085796773,
 -0.08524026429509902,
 0.012489654595518386,
 -0.012896382282060855,
 0.04017099812262208,
 0.021963608978347808,
 -0.013780635869431214,
 -0.007941209351691242,
 -0.03592801144301233,
 0.008422616477901828,
 0.09935918716121218,
 -0.004453782698208034,
 0.015592152011516296,
 -0.003557902555598779,
 -0.0015856982726722437,
 0.022275390405339792,
 0.

In [ ]:
# Test: if we have $100 at the beginning of 2012, how much we would have in 11 yr by idnetifying "Bullish Engulfing" as a buying signal.
# Strategy: buy AAPL the day after "Bullish Engulfing", hold it for ten trading days then sell all the position.
original_money = 100

for i in returns:
  original_money = original_money * (1+i)
print("we would have",round(original_money,2),"by the end of 2022.")
print("The yearly return is :",round(((original_money/100)**(1/11)-1)*100,2),"%.")

we would have 538.55 by the end of 2022.
The yearly return is : 16.54 %.


Combine the above code into a general def.

In [ ]:
def return_calculator(buying_sig='bullish_engulfing',selling_sig='bearish_engulfing'):
  # Filter the dataframe to only include rows with a bullish engulfing pattern
  buying_df = df[df[buying_sig] == True]

  # Filter the dataframe to only include rows with a bearish engulfing pattern
  selling_df = df[df[selling_sig] == True]

  # Create an empty list to store the returns
  returns = []

  # Loop through each bullish engulfing pattern
  for i in range(len(buying_df)):
    # Find the next bearish engulfing pattern after the bullish engulfing pattern
    next_selling_df = selling_df[selling_df['TimeStamp'] > buying_df.iloc[i]['TimeStamp']].head(1)
    if len(next_selling_df) > 0:
        # Calculate the return from the bullish engulfing closing price to the bearish engulfing opening price
        buy_price = buying_df.iloc[i]['close']
        sell_price = next_selling_df.iloc[0]['close']
        cum_return = (sell_price - buy_price) / buy_price
        returns.append(cum_return)
  
  original_money = 100

  for i in returns:
    original_money = original_money * (1+i)
  
  return "{:.2%}".format(round(((original_money/100)**(1/11)-1),4))

In [ ]:
return_calculator(buying_sig='bullish_engulfing',selling_sig='bearish_engulfing')

'16.54%'

Test the return of each pair.

In [ ]:
buying_points = ["bullish_hammer","bullish_kicker","bullish_engulfing"]
selling_points = ["bearish_hammer","bearish_kicker","bearish_engulfing"]

results_df = pd.DataFrame(columns=['buying_point', 'selling_point', 'returns'])

for buying_point in buying_points:
  for selling_point in selling_points:
    results_df = results_df.append({'buying_point': buying_point, 'selling_point': selling_point, 
                                    'returns': return_calculator(buying_sig=buying_point,selling_sig=selling_point)}, ignore_index=True)

results_df

,buying_point,selling_point,returns
0,bullish_hammer,bearish_hammer,0.00%
1,bullish_hammer,bearish_kicker,0.00%
2,bullish_hammer,bearish_engulfing,0.00%
3,bullish_kicker,bearish_hammer,-77.39%
4,bullish_kicker,bearish_kicker,7.37%
5,bullish_kicker,bearish_engulfing,27.11%
6,bullish_engulfing,bearish_hammer,-54.45%
7,bullish_engulfing,bearish_kicker,11.83%
8,bullish_engulfing,bearish_engulfing,16.54%


Therefore, for AAPL it is best to use "bullish_kicker" as the buying point and "bearish_engulfing" as the selling point.